In [ ]:
import torch
import librosa
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset Loader
class ASVspoofDataset(Dataset):
    def __init__(self, data_dir, target_sample_rate=8000, fixed_length=32000):  
        self.data_dir = data_dir
        self.files = []
        self.labels = []
        self.target_sample_rate = target_sample_rate
        self.fixed_length = fixed_length  

        real_dir = os.path.join(data_dir, "real")
        fake_dir = os.path.join(data_dir, "fake")

        if os.path.exists(real_dir):
            real_files = [os.path.join(real_dir, f) for f in os.listdir(real_dir) if f.lower().endswith('.wav')]
            self.files.extend(real_files)
            self.labels.extend([1] * len(real_files))  
        
        if os.path.exists(fake_dir):
            fake_files = [os.path.join(fake_dir, f) for f in os.listdir(fake_dir) if f.lower().endswith('.wav')]
            self.files.extend(fake_files)
            self.labels.extend([0] * len(fake_files))  
        
        if not self.files:
            print(f"Warning: No .wav files found in {data_dir}")
        else:
            print(f"Found {len(self.files)} .wav files in {data_dir}")

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path = self.files[idx]
        waveform, sample_rate = librosa.load(file_path, sr=None)  
        waveform = librosa.resample(waveform, orig_sr=sample_rate, target_sr=self.target_sample_rate)  
        
        if len(waveform) < self.fixed_length:
            pad_length = self.fixed_length - len(waveform)
            waveform = np.pad(waveform, (0, pad_length), mode='constant')
        else:
            waveform = waveform[:self.fixed_length]

        waveform = torch.tensor(waveform, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return waveform, label

# Load dataset
dataset = ASVspoofDataset(r"C:\Users\Lenovo\archive\train")

if len(dataset) > 0:
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Simple RawNet-like model
    class RawNet(nn.Module):
        def __init__(self):
            super(RawNet, self).__init__()
            self.conv1 = nn.Conv1d(1, 64, kernel_size=3, stride=1, padding=1)
            self.relu = nn.ReLU()
            self.fc = nn.Linear(64, 1)

        def forward(self, x):
            x = self.conv1(x)
            x = self.relu(x)
            x = torch.mean(x, dim=-1)
            x = self.fc(x)
            return torch.sigmoid(x)

    # Model initialization
    model = RawNet().to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop with accuracy calculation
    for epoch in range(5):
        correct = 0
        total = 0

        for batch in dataloader:
            waveforms, labels = batch
            waveforms = waveforms.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(waveforms.unsqueeze(1))  

            # ✅ Compute loss
            loss = criterion(outputs, labels.view(-1, 1))  
            loss.backward()
            optimizer.step()

            # ✅ Compute accuracy
            predictions = (outputs >= 0.5).float()  # Convert probabilities to 0 or 1
            correct += (predictions == labels.view(-1, 1)).sum().item()
            total += labels.size(0)

        accuracy = correct / total * 100  # Convert to percentage
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%")

    print("Training complete!")

else:
    print("Dataset is empty. Training skipped.")


Found 13185 .wav files in C:\Users\Lenovo\archive\train
Epoch 1, Loss: 0.2246, Accuracy: 80.89%
Epoch 2, Loss: 0.1836, Accuracy: 82.06%
Epoch 3, Loss: 0.1466, Accuracy: 83.36%
Epoch 4, Loss: 0.2272, Accuracy: 83.70%
